# Basic Levelness Annotation Creation for a Crowd Sourced Gold Standard

BL annotation could range from 0-1
0, not basic level
1, surely basic level
anything inbetween gives estimate to bl likelihood

Steps:

* DF per 'annotator', create hypothetical binary annotation (bl, or not) 
 - select fastest times in both true/false type triples
 - is same category + accurate: annotate with 1
 - is different categories for fastest times + accurate: both get 0.5
 - is fastest time inaccurate, select next fastest time, give 0.5
 - two fastest times inaccurate, select remaining, give 0.3
 - all inaccurate, select superordinate, give 0.1

* Create new GOLD STANDARD data frame (per branch) like hollink (compatibility):
  -> | synset | glossary | depth in hierarchy | direct hypernyms | direct hyponyms | accuracy | blness |

* Per synset, combine blness measure (mean?), add mean accuracy
* I would still be interested in an inter-annotator measure that reveals groups of 'similarly' rating annotators. Could use to split gold standard in two w.r.t. groups of ppl.


## Data Preparation

Import Experiment Results.

Create data frame to be annotated (with correct WordNet synsets)

-> | synset | gloss | hierarchy_depth | expert_certainty | blness | accuracy |

In [19]:
from nltk.corpus import wordnet as wn
import pandas as pd
import os, csv

# specify experiment results folder in question
experiment_data_folder = os.path.abspath("../Past_experiments/Pilot2/collected_data/experiment_data")

# specify experiment 'project file'
experiment_project = pd.read_csv(os.path.abspath("../Past_experiments/Pilot2/Rosch7_Thesis_Experiment/Thesis_Experiment.csv"))

experiment_project.head()


,hypernym,bl,bl_name,bl_certainty,hyponym,hypernym_def,bl_def,hyponym_def,hyponym_img,cont_flag
0,hand_tool.n.01,awl.n.01,awl,True,bradawl.n.01,missing,a pointed tool for marking surfaces or for pun...,an awl for making small holes for brads or sma...,bradawl.n.01.png,True
1,hand_tool.n.01,file.n.04,file,True,nailfile.n.01,missing,a steel hand tool with small sharp teeth on so...,a small flat file for shaping the nails,nailfile.n.01.png,False
2,hand_tool.n.01,hammer.n.02,hammer,True,carpenter's_hammer.n.01,missing,a hand tool with a heavy rigid head and a hand...,a hammer with a cleft at one end for pulling n...,carpenter's_hammer.n.01.png,False
3,hand_tool.n.01,can_opener.n.01,can_opener,True,church_key.n.01,missing,a device for cutting cans open,can opener that has a triangular pointed end t...,church_key.n.01.png,False
4,hand_tool.n.01,plane.n.05,plane,True,smooth_plane.n.01,missing,a carpenter's hand tool with an adjustable bla...,a small plane for finish work,smooth_plane.n.01.png,False


In [41]:
def get_depth_wn(syns):
    # to do
    return 1

In [47]:
#collect array of branches as names of data frames
branches = experiment_project["hypernym"].unique()
gstandards = dict()

for branch in branches:
     subdf = experiment_project.loc[experiment_project['hypernym'] == branch]
     
     # writing the superordinate category row
     syns = [branch]
     gloss = [wn.synset(branch).definition()]
     hd = [get_depth_wn(branch)]
     ecert = [True]
     bls = [0]
     accs = [1]

     for i, row in subdf.iterrows():
          bl = row['bl']
          sub = row['hyponym']

          # writing the bl category row
          syns.append(bl)
          gloss.append(wn.synset(bl).definition())
          hd.append(get_depth_wn(bl))
          certainty = row['bl_certainty']
          ecert.append(certainty)

          if certainty:
               bls.append(1)
               accs.append(1)
          else:
               bls.append(0.7)
               accs.append(0.5)

          # writing the subordinate category row
          syns.append(sub)
          gloss.append(wn.synset(sub).definition())
          hd.append(get_depth_wn(sub))
          certainty = row['bl_certainty']
          ecert.append(certainty)

          if certainty:
               bls.append(0)
               accs.append(1)
          else:
               bls.append(0.3)
               accs.append(0.5)


     d = {'synset': syns,
         'glossary': gloss,
         'hierarchy_depth': hd,
         'expert_certainty': ecert,
         'blness': bls,
         'accuracy': accs}
     gstandards[branch] = pd.DataFrame(data=d)

print(gstandards[branch].head(5))

             synset                                           glossary  \
0     clothing.n.01  a covering designed to be worn on a person's body   
1      trouser.n.01  (usually in the plural) a garment extending fr...   
2  sweat_pants.n.01  loose-fitting trousers with elastic cuffs; wor...   
3      sweater.n.01  a crocheted or knitted garment covering the up...   
4   turtleneck.n.01  a sweater or jersey with a high close-fitting ...   

   hierarchy_depth  expert_certainty  blness  accuracy  
0                1              True     0.0       1.0  
1                1              True     1.0       1.0  
2                1              True     0.0       1.0  
3                1              True     1.0       1.0  
4                1              True     0.0       1.0  


In [76]:
participant_files = [os.path.join(experiment_data_folder, file) for file in os.listdir(experiment_data_folder)]
participant_dfs = [pd.read_csv(file, sep=" ",names=['index','synset','lvl', 'img', 'type', 'accuracy', 'rt']) for file in participant_files]
dfs_for_rating = [df[['synset','blness','accuracy']] for df in [gstandards[br] for br in branches]]

for df in dfs_for_rating:
    for p_df in participant_dfs:
        for i, row in df.iterrows():
            syn = row['synset']
            results = p_df.loc[syn == p_df['synset']]
            print(results)
            break

Empty DataFrame
Columns: [index, synset, lvl, img, type, accuracy, rt]
Index: []
Empty DataFrame
Columns: [index, synset, lvl, img, type, accuracy, rt]
Index: []
Empty DataFrame
Columns: [index, synset, lvl, img, type, accuracy, rt]
Index: []
Empty DataFrame
Columns: [index, synset, lvl, img, type, accuracy, rt]
Index: []
Empty DataFrame
Columns: [index, synset, lvl, img, type, accuracy, rt]
Index: []
Empty DataFrame
Columns: [index, synset, lvl, img, type, accuracy, rt]
Index: []
Empty DataFrame
Columns: [index, synset, lvl, img, type, accuracy, rt]
Index: []
Empty DataFrame
Columns: [index, synset, lvl, img, type, accuracy, rt]
Index: []
Empty DataFrame
Columns: [index, synset, lvl, img, type, accuracy, rt]
Index: []
Empty DataFrame
Columns: [index, synset, lvl, img, type, accuracy, rt]
Index: []
Empty DataFrame
Columns: [index, synset, lvl, img, type, accuracy, rt]
Index: []
Empty DataFrame
Columns: [index, synset, lvl, img, type, accuracy, rt]
Index: []
Empty DataFrame
Columns: [in